# To send personalized table as attachment to ADM vendors

### SOP for using this notebook to send personalized table from a dump file as attachment to ADM vendors
1. Go to https://myaccount.google.com/apppasswords and create new app profile & password associated with your personal flipkart account. Ensure you have 2 step authorization with your flipkart account.
2. The password has to be updated in Step 5 along with your email ID
3. The updated Vendor list along with their email IDs has to be stored in an .xlsx file locally & the path to the file has to be updated in Step 2.
4. The respective vendor files will be processed in a local folder for processing. The folder path has to be updated in Step 4.
5. The file has to be executed in the order mentioned in this file from Step 1 to Step 5.
6. The file contains paths used during testing. Please update before using.

## 1. Install necessary packages in python

In [1]:
# Install packages
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import datetime
from datetime import datetime
from datetime import timedelta
import pandas as pd
import seaborn as sns
import requests
import os
from pathlib import Path

#for sending mail
import smtplib
#import imghdr
from email.message import EmailMessage
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import glob

## 2. Read Vendor names & their email IDs from local excel file

In [2]:
df = pd.read_csv(r'C:\\Users\\prajwal.d1\\Desktop\\Pendency\\ODH Model pincode Tracker.csv')
df1 = df[['ODH/MDH Name','Type','Vendor','Zone','State']]
Map = df1.drop_duplicates()
Map.rename(columns={'Vendor':'Vendor Name'},inplace=True)
Map['Vendor Name'] = Map['Vendor Name'].astype(str).str.upper()
Map['ODH/MDH Name'] = Map['ODH/MDH Name'].astype(str).str.upper()
Map

,ODH/MDH Name,Type,Vendor Name,Zone,State
0,SASTHURAVOORODH_THO,ODH,SAS,SOUTH,KERALA
1,RJSCIMPHALODH_IMP,ODH,RJSC,EAST,MANIPUR
2,IMPHALSPLITMDH_IMP,MDH in Bin,RJSC,EAST,MANIPUR
3,ELASTICRUNBHAWANIPURODH_KOL,ODH,ELASTICRUN,EAST,WEST BENGAL
4,CABTGARIAHATODH_KOL,ODH,LOADSHARE,EAST,WEST BENGAL
...,...,...,...,...,...
2393,UCCMDH_SHL,MDH,LIGHTWINGS,EAST,MEGHALAYA
2394,JHALDAHUB_JAA,ODH,LOADSHARE,EAST,WEST BENGAL
2395,GANGADRAMDH_KDO,MDH,SHADOWFAX,WEST,GUJARAT
2396,MAWSYNRAMMDH_SHL,MDH,LIGHTWINGS,EAST,MEGHALAYA


In [3]:
# Read each file separately with explicit file paths
OBD_Adh = pd.read_excel(r"C:\Users\prajwal.d1\Desktop\Fraud_Data\OBD_W14.xlsx") #NM
Label = pd.read_excel(r"C:\Users\prajwal.d1\Desktop\Fraud_Data\Relabel_Send22.xlsx") #NM
PRC = pd.read_excel(r"C:\Users\prajwal.d1\Desktop\Fraud_Data\PRC_Send21.xlsx") #NM
PickupAccuracy = pd.read_excel(r'C:\\Users\\prajwal.d1\\Desktop\\Fraud_Data\\PA15A22A.xlsx', engine="openpyxl", sheet_name="Data")
FST = pd.read_excel(r"C:\Users\prajwal.d1\Desktop\Fraud_Data\FST.xlsx")
POD = pd.read_excel(r'C:\\Users\\prajwal.d1\\Desktop\\Fraud_Data\\POD14A.xlsx', engine="openpyxl", sheet_name="EKL View")
ExternalizationRVP = pd.read_excel(r'C:\\Users\\prajwal.d1\\Desktop\\Fraud_Data\\EXTRVP15A.xlsx', engine="openpyxl", sheet_name="Raw")
ExternalizationRTO = pd.read_excel(r'C:\\Users\\prajwal.d1\\Desktop\\Fraud_Data\\EXTRTO15A.xlsx', engine="openpyxl", sheet_name="Raw")

In [4]:
POD.columns

Index(['TID ', 'RTO Created Date ', 'Type ', 'Approved claim Amount ',
       'reason', 'SPF Payout reason ', 'seller_id', 'seller_display_name',
       'Status ', 'Last Updated Date ', 'Ageing ', 'Age Bucket ',
       'First Assigned Hub ', 'Hub Name ', 'Mapped Hub ', 'GM ', 'Zone',
       'Asset Type ', 'Asset Type New'],
      dtype='object')

In [5]:
OBD_Adh['parent_lm_hub'] = OBD_Adh['parent_lm_hub'].astype(str).str.upper()
Label['ODH/MDH Name'] = Label['ODH/MDH Name'].astype(str).str.upper()
PRC['ODH/MDH Name'] = PRC['ODH/MDH Name'].astype(str).str.upper()
PickupAccuracy['Hub name'] = PickupAccuracy['Hub name'].astype(str).str.upper()
FST['Hub Name'] = FST['Hub Name'].astype(str).str.upper()
POD['HubName'] = POD['Hub Name '].astype(str).str.upper()
ExternalizationRVP['dh_name'] = ExternalizationRVP['dh_name'].astype(str).str.upper()
ExternalizationRTO['dh_name'] = ExternalizationRTO['dh_name'].astype(str).str.upper()

In [6]:
PickupAccuracy = pd.merge(PickupAccuracy, Map[['ODH/MDH Name','Vendor Name']], how='left', left_on = 'Hub name', right_on = 'ODH/MDH Name')
PickupAccuracy=PickupAccuracy.drop_duplicates()
PickupAccuracy.drop(columns = ['ODH/MDH Name'],inplace = True)

FST = pd.merge(FST, Map[['ODH/MDH Name','Vendor Name']], how='left', left_on = 'Hub Name', right_on = 'ODH/MDH Name')
FST=FST.drop_duplicates()
FST.drop(columns = ['ODH/MDH Name'],inplace = True)

ExternalizationRVP = pd.merge(ExternalizationRVP, Map[['ODH/MDH Name','Vendor Name']], how='left', left_on = 'dh_name', right_on = 'ODH/MDH Name')
ExternalizationRVP=ExternalizationRVP.drop_duplicates()
ExternalizationRVP.drop(columns = ['ODH/MDH Name'],inplace = True)

ExternalizationRTO = pd.merge(ExternalizationRTO, Map[['ODH/MDH Name','Vendor Name']], how='left', left_on = 'dh_name', right_on = 'ODH/MDH Name')
ExternalizationRTO=ExternalizationRTO.drop_duplicates()
ExternalizationRTO.drop(columns = ['ODH/MDH Name'],inplace = True)

POD = pd.merge(POD, Map[['ODH/MDH Name','Vendor Name']], how='left', left_on = 'Hub Name ', right_on = 'ODH/MDH Name')
POD=POD.drop_duplicates()
POD.drop(columns = ['ODH/MDH Name'],inplace = True)



In [7]:
# Reading a database of vendors & email ids
VendorContact = pd.read_excel(r'C:\\Users\\prajwal.d1\\Desktop\\Pendency\\Vendor_Mail.xlsx', dtype=str)

In [8]:
VendorNames = VendorContact["Vendor Name"].tolist()
VendorMail = VendorContact["Vendor Mail"].tolist()

## 3. Creating file directory & clear old data

In [9]:
# Create a folder and update path here
DownloadPath = "C:\\Users\\prajwal.d1\\Desktop\\Fraud_Data\\Fraud_Reports_Files_2"

In [10]:
# Remove all files in the specified folder
os.chdir(f"{DownloadPath}")
all_files = os.listdir()

for f in all_files:
    os.remove(f)

## 4. Processing data for respective vendors

In [11]:
OBD_Adh.columns

Index(['Business Unit', 'id', 'brand', 'product_title', 'vertical',
       'is_alpha_seller', 'service_profile', 'seller_id', 'pincode',
       'tracking_id', 'prexo_flag', 'order_date', 'status', 'o2o_flag',
       'delivered_date_time', 'fulfill_item_unit_deliver_actual_time',
       'fulfillment_done_initial_customer_promised_date',
       'fulfill_item_unit_expected_delivery_date_internal', 'unit_status',
       'unit_latest_status', 'fulfill_item_unit_out_for_delivery_actual_time',
       'fwd_fe_profile_id', 'hub_type', 'Vendor Name ', 'hub_name',
       'parent_lm_hub', 'hub_zone', 'fe_employee_id', 'fe_line_of_business',
       'fe_phone_number', 'fe_name', 'OBD Image validation', 'Not Okay Reason',
       'GM Name', 'Type of hub', 'RM Name', 'AM Name', 'State'],
      dtype='object')

In [12]:
# Create separate files for vendors
attachment_filename_fwd = []

for x in range(len(VendorNames)):
    VendorData_OBD_Adh = OBD_Adh.loc[OBD_Adh['Vendor Name '] == VendorNames[x]]
    VendorData_Label = Label.loc[Label['Vendor Name'] == VendorNames[x]]
    VendorData_PRC = PRC.loc[PRC['Vendor Name'] == VendorNames[x]]
    VendorData_PickupAccuracy = PickupAccuracy.loc[PickupAccuracy['Vendor Name'] == VendorNames[x]]
    VendorData_FST = FST.loc[FST['Vendor Name'] == VendorNames[x]]
    VendorData_POD = POD.loc[POD['Vendor Name'] == VendorNames[x]]
    VendorData_ExternalizationRVP = ExternalizationRVP.loc[ExternalizationRVP['Vendor Name'] == VendorNames[x]]
    VendorData_ExternalizationRTO = ExternalizationRTO.loc[ExternalizationRTO['Vendor Name'] == VendorNames[x]]

    filename=VendorNames[x]+"_Fraud_Reports.xlsx"
    with pd.ExcelWriter(filename) as writer:
        VendorData_OBD_Adh.to_excel(writer, sheet_name='OBD_Adh_W14',index=False)
        VendorData_Label.to_excel(writer, sheet_name='Relabel_Generation_23A',index=False)
        VendorData_PRC.to_excel(writer, sheet_name='PRC_21A',index=False)
        VendorData_PickupAccuracy.to_excel(writer, sheet_name='PickupAccuray_15A22A',index=False)
        VendorData_FST.to_excel(writer, sheet_name='FST_Responces',index=False)
        VendorData_POD.to_excel(writer, sheet_name='POD_14A',index=False)
        VendorData_ExternalizationRVP.to_excel(writer, sheet_name='ExternalizationRVP_15A',index=False)
        VendorData_ExternalizationRTO.to_excel(writer, sheet_name='ExternalizationRTO_15A',index=False)
        
    attachment_filename_fwd.append(VendorNames[x]+"_Fraud_Reports.xlsx")


## 5. Send E-mail to vendor with their respective attachment

In [13]:
# Enter credentials of your account
EMAIL_ADDRESS = 'prajwal.d1@flipkart.com'
EMAIL_PASSWORD = 'yvzokolqwsbyjbqo'

In [14]:
for y in range(len(VendorNames)):
    msg = EmailMessage()
    msg['Subject'] = "Fraud Reports – 23 April'25 Part-2 "+VendorNames[y]
    msg['From'] = EMAIL_ADDRESS
    msg['To'] = VendorMail[y]
    msg.set_content("""Dear Partner,

    Do find attached the various Fraud Data / OBD-CV Adh reports along with timelines for your reference. 
    Kindly review the details below and take necessary actions:


    1. OBD Non Adh:  
    
   - OBD Non Adh has been mapped at the TID/FE level. 
     Kindly review the data and ensure that HI's conduct training sessions for FEs with high non-adherence. 
     Repeated instances of non-adherence will result in blacklisting.  
     

    2. Pickup Accuracy:
   
   - Image accuracy reports at the FE level have been attached. 
     Please ensure that adherence is driven in cases of non-adherence, as these will be part of the recovery scope if shipments are involved in fraud.  
     

   3. Externalization Fraud Loss:  
   
   - There is a rising trend of DNU fraud being externalized. 
     Please review the TIDs and take necessary action against WMs marking shipments as undeliverable incorrectly.


   4. FST/OBD

    - If the shipments are falling under OBD and FST, 
      please check the ODH - FST/OBD Fail Investigations form. 
     Also, do find the response raw dump.
     The Google Form link - https://docs.google.com/forms/d/e/1FAIpQLSfrjou3fLKIhff_dw51B4agH3iyswco6gvV8HL7TpfSi0d41Q/viewform
     
     
     

      Actionable Points:
          - Review the fraud reports and share your findings.  
          - Work on improving CV and OBD Non Adh at the Hub/FE level.
          
         
- NOTE: Issue warning letters to WMs where non-adherence is found in more than five shipments. Continued non-adherence will result in blacklisting from the system.  
    
Thanks & Regards,
Prajwal | 
7975770738""")
    attachment_filename_fwd=(VendorContact.loc[VendorContact['Vendor Name'] == VendorNames[y],"Vendor Name"]+"_Fraud_Reports.xlsx").tolist()
    files1=[attachment_filename_fwd[0]]
    for file in files1:
        with open(file,'rb') as f:
            file_data1 = f.read()
            file_name1 = f.name
        msg.add_attachment(file_data1, maintype='application', subtype='octet-stream', filename=file_name1)
    #PDF=Temp.loc[Temp['File']=='Gate_pass_Format.pdf',"File"].tolist()
    #files2 =[PDF[0]]
    #for file in files2:
     #   with open(file,'rb') as f:
      #      file_data2 = f.read()
       #     file_name2 = f.name
        #msg.add_attachment(file_data2, maintype='application', subtype='octet-stream', filename=file_name2)
    # with smtplib.SMTP('smtp.gmail.com', 587, timeout=30) as smtp:
     # smtplib.SMTP('smtp.gmail.com', 587, timeout=60) as smtp:
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp: 
        smtp.login(EMAIL_ADDRESS,EMAIL_PASSWORD)
        smtp.send_message(msg)    

In [15]:
#FwdDump.head()

In [ ]:
# Remove all files in the specified folder
os.chdir(f"{DownloadPath}")
all_files = os.listdir()
os.getcwd()

In [ ]:
(VendorContact.loc[VendorContact['Vendor Name'] == VendorNames[y],"Vendor Name"]+"_RTO_RVP_UT_28.xlsx")

In [ ]:
#   10. Relabeling Data : 1st Feb'25 to 03rd Feb'25
#       * There is a growing trend where fraudulent transactions are being relabeled as legitimate, which can significantly affect the accuracy of our fraud detection models.
#
#   11. Externalization Fraud loss - 1stJan'25 to 28thJan'25  
#       * There is High trend of DNU fraud is Externalization - Pl go through the TIDs and take action on WM Marking Undeli
#
#10. Externalization Fraud loss - 1stDec'24 to 31stDec'24  
#       * There is High trend of DNU fraud is Externalization - Pl go through the TIDs and take action on WM Marking Undel
#11. Externalization Fraud loss - 1stJan'25 to 07thJan'25  
#       * There is High trend of DNU fraud is Externalization - Pl go through the TIDs and take action on WM Marking Undel
